# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse        Shipments  \
0  Jun 27 2022 11:04AM  240518        10       0085799585   
1  Jun 27 2022 11:04AM  240518        10       0085799765   
2  Jun 27 2022 10:48AM  240516        20  SSNA-2173691942   
3  Jun 27 2022 10:45AM  240515        16   EMERSA-8188293   
4  Jun 27 2022 10:31AM  240513        10        PRF-32207   
5  Jun 27 2022 10:31AM  240513        10        PRF-32208   
6  Jun 27 2022 10:31AM  240513        10        PRF-32217   
7  Jun 27 2022 10:31AM  240513        10        PRF-32223   
8  Jun 27 2022 10:31AM  240513        10        PRF-32227   
9  Jun 27 2022 10:31AM  240513        10        PRF-32230   

                           Customer ShipmentStatus  
0                 ISDIN Corporation       Released  
1                 ISDIN Corporation       Released  
2  Sartorius Stedim North America         Released  
3              Emersa Waterbox, LLC       Released  
4                           Bio-PRF       Released  
5                           Bio-PRF       Released  
6                           Bio-PRF       Released  
7                           Bio-PRF       Released  
8                           Bio-PRF       Released  
9                           Bio-PRF       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  240512       Released         12
40  240513       Released         11
41  240515       Released          1
42  240516       Released          1
43  240518       Released          2

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
240512                NaN      12.0
240513                NaN      11.0
240515                NaN       1.0
240516                NaN       1.0
240518                NaN       2.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
240455                0.0       4.0
240463               41.0       6.0
240464                0.0       1.0
240467                0.0      49.0
240469                0.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
240455                  0         4
240463                 41         6
240464                  0         1
240467                  0        49
240469                  0         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               240455          0         4
1               240463         41         6
2               240464          0         1
3               240467          0        49
4               240469          0         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               240455                  4
1               240463        41        6
2               240464                  1
3               240467                 49
4               240469                  3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0   Jun 27 2022 11:04AM  240518        10   
2   Jun 27 2022 10:48AM  240516        20   
3   Jun 27 2022 10:45AM  240515        16   
4   Jun 27 2022 10:31AM  240513        10   
15  Jun 27 2022 10:23AM  240512        10   
27  Jun 27 2022 10:18AM  240495        10   
43  Jun 27 2022 10:18AM  240511        20   
44  Jun 27 2022 10:13AM  240507        15   
45  Jun 27 2022 10:12AM  240506        15   
49  Jun 27 2022 10:10AM  240504        15   

                                 Customer  
0                       ISDIN Corporation  
2        Sartorius Stedim North America    
3                    Emersa Waterbox, LLC  
4                                 Bio-PRF  
15                       Methapharm, Inc.  
27                      ISDIN Corporation  
43       Sartorius Stedim North America    
44                 Mizner Bioscience, LLC  
45  Carwin Pharmaceutical Associates, LLC  
49                   Person & Covey, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  Jun 27 2022 11:04AM  240518        10   
1  Jun 27 2022 10:48AM  240516        20   
2  Jun 27 2022 10:45AM  240515        16   
3  Jun 27 2022 10:31AM  240513        10   
4  Jun 27 2022 10:23AM  240512        10   
5  Jun 27 2022 10:18AM  240495        10   
6  Jun 27 2022 10:18AM  240511        20   
7  Jun 27 2022 10:13AM  240507        15   
8  Jun 27 2022 10:12AM  240506        15   
9  Jun 27 2022 10:10AM  240504        15   

                                Customer Executing Released  
0                      ISDIN Corporation                  2  
1       Sartorius Stedim North America                    1  
2                   Emersa Waterbox, LLC                  1  
3                                Bio-PRF                 11  
4                       Methapharm, Inc.                 12  
5                      ISDIN Corporation                 16  
6       Sartorius Stedim North America                    1  
7                 Mizner Bioscience, LLC         1           
8  Carwin Pharmaceutical Associates, LLC                  4  
9                   Person & Covey, Inc.                 10

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  Jun 27 2022 11:04AM  240518        10   
1  Jun 27 2022 10:48AM  240516        20   
2  Jun 27 2022 10:45AM  240515        16   
3  Jun 27 2022 10:31AM  240513        10   
4  Jun 27 2022 10:23AM  240512        10   
5  Jun 27 2022 10:18AM  240495        10   
6  Jun 27 2022 10:18AM  240511        20   
7  Jun 27 2022 10:13AM  240507        15   
8  Jun 27 2022 10:12AM  240506        15   
9  Jun 27 2022 10:10AM  240504        15   

                                Customer Released Executing  
0                      ISDIN Corporation        2            
1       Sartorius Stedim North America          1            
2                   Emersa Waterbox, LLC        1            
3                                Bio-PRF       11            
4                       Methapharm, Inc.       12            
5                      ISDIN Corporation       16            
6       Sartorius Stedim North America          1            
7                 Mizner Bioscience, LLC                  1  
8  Carwin Pharmaceutical Associates, LLC        4            
9                   Person & Covey, Inc.       10

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun 27 2022 11:04AM  240518        10                 ISDIN Corporation   
1  Jun 27 2022 10:48AM  240516        20  Sartorius Stedim North America     
2  Jun 27 2022 10:45AM  240515        16              Emersa Waterbox, LLC   
3  Jun 27 2022 10:31AM  240513        10                           Bio-PRF   
4  Jun 27 2022 10:23AM  240512        10                  Methapharm, Inc.   

  Released Executing  
0        2            
1        1            
2        1            
3       11            
4       12

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                          Customer  \
0  Jun 27 2022 11:04AM  240518        10                 ISDIN Corporation   
1  Jun 27 2022 10:48AM  240516        20  Sartorius Stedim North America     
2  Jun 27 2022 10:45AM  240515        16              Emersa Waterbox, LLC   
3  Jun 27 2022 10:31AM  240513        10                           Bio-PRF   
4  Jun 27 2022 10:23AM  240512        10                  Methapharm, Inc.   

   Released  Executing  
0       2.0        NaN  
1       1.0        NaN  
2       1.0        NaN  
3      11.0        NaN  
4      12.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun 27 2022 11:04AM  240518        10                 ISDIN Corporation   
1  Jun 27 2022 10:48AM  240516        20  Sartorius Stedim North America     
2  Jun 27 2022 10:45AM  240515        16              Emersa Waterbox, LLC   
3  Jun 27 2022 10:31AM  240513        10                           Bio-PRF   
4  Jun 27 2022 10:23AM  240512        10                  Methapharm, Inc.   

   Released  Executing  
0       2.0        0.0  
1       1.0        0.0  
2       1.0        0.0  
3      11.0        0.0  
4      12.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3847804     391.0       20.0
12          721432       9.0       41.0
15         1443017      46.0       18.0
16          721463       3.0        0.0
19          480949       5.0        0.0
20          961998      21.0        0.0
21         1202492       5.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3847804     391.0       20.0
1        12   721432       9.0       41.0
2        15  1443017      46.0       18.0
3        16   721463       3.0        0.0
4        19   480949       5.0        0.0
5        20   961998      21.0        0.0
6        21  1202492       5.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     391.0       20.0
1        12       9.0       41.0
2        15      46.0       18.0
3        16       3.0        0.0
4        19       5.0        0.0
5        20      21.0        0.0
6        21       5.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  391.0
1        12  Released    9.0
2        15  Released   46.0
3        16  Released    3.0
4        19  Released    5.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    15   16   19    20   21
Status                                           
Executing   20.0  41.0  18.0  0.0  0.0   0.0  0.0
Released   391.0   9.0  46.0  3.0  5.0  21.0  5.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    15   16   19    20   21
0          Executing   20.0  41.0  18.0  0.0  0.0   0.0  0.0
1           Released  391.0   9.0  46.0  3.0  5.0  21.0  5.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    15   16   19    20   21
0  Executing   20.0  41.0  18.0  0.0  0.0   0.0  0.0
1   Released  391.0   9.0  46.0  3.0  5.0  21.0  5.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()